In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [103]:
month_code = '11'
month_day = 30

In [105]:
file_discern_name = month_code + '.xlsx'
ncd_app = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_app24'+file_discern_name)
ncd_att = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_att24'+file_discern_name)
ncd_box = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_box24'+file_discern_name)
ncd_corlt = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corlt24'+file_discern_name)
ncd_corstd = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstd24'+file_discern_name)
ncd_corstdp = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstdp24'+file_discern_name)
ncd_cort = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cort24'+file_discern_name)
ncd_drv = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_drv24'+file_discern_name)
#ncd_cnt = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cnt24'+file_discern_name)
ncd_fin = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_fin24'+file_discern_name)
ncd_pay = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_pay24'+file_discern_name)

ncd_app['plan'] = 'APP_STD'
ncd_att['plan'] = 'ATT_STD'
ncd_corlt['plan'] = 'COR_LT'
ncd_corstd['plan'] = 'COR_STD'
ncd_corstdp['plan'] = 'COR_STD_P'
ncd_cort['plan'] = 'COR_T'
ncd_drv['plan'] = ''
ncd_box['plan'] = 'BOX'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 체험판','plan'] = 'DRV_T'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Lite','plan'] = 'DRV_LT'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Standard','plan'] = 'DRV_STD'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Standard Plus','plan'] = 'DRV_STD_P'
#ncd_cnt['plan'] = ''
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 체험판','plan'] = 'CNT_T'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Lite','plan'] = 'CNT_LT'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Business','plan'] = 'CNT_BZ'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Team','plan'] = 'CNT_TM'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Enterprise','plan'] = 'CNT_EP'
ncd_fin['plan'] = 'FIN_STD'
ncd_pay['plan'] = 'PAY_STD'
ncd_box['사용량'] = ncd_box['상품사이즈']
ncd_box['사용량'] = ncd_box['사용량'].fillna(0)

ncd = pd.concat([ncd_app, ncd_att, ncd_box, ncd_corlt, ncd_corstd, ncd_corstdp, ncd_cort, ncd_drv, ncd_fin, ncd_pay])
sales_expectation = []
recent_month = pd.read_excel('/Users/user/Documents/업무문서/ncd'+str(int(month_code)-1)+'_sales_expectation.xlsx')
pt = recent_month[recent_month['회원그룹코드'] == 'PATR']
pt = pt.copy()
pt.loc[:, '파트너할인율'] = 0.0
pt.loc[:, '파트너할인율'] = 1 - (pt['예상월매출'] / pt['예상월요금'])
pt = pt[['회원번호', '구분3', '파트너사회원번호','파트너할인율']]
ncd_copy = ncd.copy()
for i in range(1,month_day+1) :
    ncd = ncd_copy[ncd_copy['미터링일시']<= (20240000+(int(month_code)*100)+i)]
    ncd = ncd.reset_index(drop=True)
    ncd = ncd[ncd['사용량'] != 0]
    ncd = ncd[ncd['요금'] != 0]
    ncd = ncd[ncd['회원그룹명'] != '내부회원']
    ncd['구분1'] = 'Business Application'
    ncd['구분2'] = 'Business Applications'
    ncd['구분3'] = ncd['인스턴스유형코드명']
    ncd['상품명'] = ncd['미터링유형코드명']
    ncd['플랫폼'] = '민간'
    ncd['리전명_y'] = '한국'
    ncd['리전번호_y'] = 1
    ncd['dayCount'] = ncd.groupby(['회원번호', '미터링유형코드명'])['회원번호'].transform('size')
    ncd['paid user 여부'] = ncd['요금'].apply(lambda x : 'trial' if x == 0 else 'paid')
    ncd['약정할인금액'] = np.where(
        (ncd['미터링유형코드명'].isin(['웍스 코어 Standard Plus'])) & (ncd['약정여부'] == 'Y'), 
        2000, 
        np.where(
            (ncd['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus'])) & (ncd['약정여부'] == 'Y'), 
            1000,
            np.where(
                (ncd['미터링유형코드명'].isin(['웍스 급여'])) & (ncd['약정여부'] == 'Y'), 
                1500,
                np.where(
                    (ncd['미터링유형코드명'].isin(['웍스 결재'])) & (ncd['약정여부'] == 'Y'), 
                    1000, 
                    np.where(
                        (ncd['미터링유형코드명'].isin(['웍스 근태','웍스 재무'])) & (ncd['약정여부'] == 'Y'), 
                        50,
                        np.where(
                            (ncd['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 코어 Standard Plus','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus','웍스 결재','웍스 근태','웍스 재무','웍스 급여'])) & (ncd['약정여부'] == 'N'), 
                            0,
                            0
                        )
                    )
                )
            )
        )
    )
    ncd['미터링일시'] = ncd['미터링일시'].astype(str)
    ncd['월별일수'] = np.where(
        ncd['미터링일시'].str[4:6].isin(['01', '03', '05', '07', '08', '10', '12']),
        31,
        np.where(
            ncd['미터링일시'].str[4:6].isin(['04', '06', '09', '11']),
            30,
            np.where(
                (ncd['미터링일시'].str[0:4].astype(int) % 4 == 0),
                29, 
                28
            )
        )
    )
    
    ncd['일요금(당일)'] = (ncd['요금']/ncd['월별일수'])*ncd['사용량']
    ncd['약정할인 후 요금'] = ncd['요금'] - ncd['약정할인금액']
    ncd['약정할인율'] = ncd['약정할인금액']/ncd['요금']
    ncd['결합할인 여부'] = 'Y'
    ncd.loc[ncd['미터링유형코드명'].isin(['웍스 근태', '웍스 재무', '웍스 급여']), '결합할인 여부'] = 'N'
    ncd['결합할인 여부'] = np.where(
        (ncd.duplicated(subset=['회원번호','미터링일시','paid user 여부','약정여부','결합할인 여부'], keep=False)) & (ncd['paid user 여부'] == 'paid') & (ncd['약정여부'] == 'Y') & (ncd['결합할인 여부'] == 'Y'),
        'Y',
        'N')
    ncd.loc[ncd['약정여부'] == 'N', '결합할인 여부'] = 'N'
    ncd['미터링일시'] = ncd['미터링일시'].astype(str)
    ncd['일매출(당일)'] = np.where(
        (ncd['결합할인 여부'] == 'Y'),
        ((ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))*0.9,
        (ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))
    ncd = ncd.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'])
    ncd['일매출(전일)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].shift(1)
    ncd['일매출 증가율'] = ((ncd['일매출(당일)'] - ncd['일매출(전일)']) / ncd['일매출(전일)'])
    ncd['누적 일매출'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].cumsum()
    ncd['월매출(당월)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].transform('sum')
    ncd['월요금(당월)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일요금(당일)'].transform('sum')
    ncd['사용량_mean'] = ncd.groupby(['회원번호', '미터링유형코드명'])['사용량'].transform('mean')
    ncd = ncd.drop(['dayCount'], axis = 1)
    ncd = ncd.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'], ascending = [True, False, True])
    ncd = ncd.reset_index(drop=True)
    ncd.loc[ncd['미터링유형코드명'].str[:5] == '웍스 드라', '청구유형코드명'] = 'WORKBOX'
    ncd.loc[ncd['미터링유형코드명'].isin(['웍스 결재','웍스 근태','웍스 재무','웍스 급여']), '청구유형코드명'] = 'WORKPLACE'
    ncd = ncd[['회원번호','파트너사회원번호','회원그룹코드','청구유형코드명','미터링유형코드명','plan','미터링일시','사용량_mean','약정여부','월매출(당월)','월요금(당월)']]
    ncd_unique = ncd.drop_duplicates(subset=['회원번호', '미터링유형코드명'], keep='first').reset_index(drop=True)
    ncd_unique['회원번호'] = ncd_unique['회원번호'].astype(int)
    ncd_unique.columns = ['회원번호','파트너사회원번호','회원그룹코드','구분3','미터링유형코드명','plan','미터링일시','사용량','약정여부','월매출(당월)','월요금(당월)']
    ncd_unique1 = ncd_unique.groupby(['회원번호','회원그룹코드','구분3'])[['월매출(당월)','월요금(당월)']].sum().reset_index()
    ncd_unique1 = pd.merge(ncd_unique1, ncd_unique[['회원번호','파트너사회원번호','구분3','plan','사용량']], how = 'left',on = ['회원번호','구분3'])
    ncd_unique = ncd_unique1
    ncd_unique = ncd_unique.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
    delete1 = pd.read_excel('/Users/user/Documents/업무문서/iima 테스트도메인.xlsx')
    delete2 = pd.read_excel('/Users/user/Documents/업무문서/Edu 제외.xlsx')
    delete3 = pd.read_excel('/Users/user/Documents/업무문서/파트너 제외.xlsx')
    delete4 = pd.read_excel('/Users/user/Documents/업무문서/수동 제외.xlsx')
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete1['ncpId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete2['ncpId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete3['partnerDomainId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete4['ncpId'])]

    merge = pd.merge(ncd_unique, pt, how='left', on=['회원번호', '구분3', '파트너사회원번호'])
    merge.columns = ['회원번호','회원그룹코드','구분3','예상월매출','예상월요금','파트너사회원번호','plan','사용량','파트너할인율']
    merge['파트너할인율'] = merge['파트너할인율'].fillna(0)
    merge.loc[merge['회원그룹코드']=='PATR','예상월매출'] = merge['예상월요금'] * (1 - merge['파트너할인율'])
    merge = merge[['회원번호','회원그룹코드','파트너사회원번호','구분3','plan','예상월매출','예상월요금','사용량']]
    merge = merge.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
    
    promotion = pd.read_excel('/Users/user/Documents/진행중 업무/웍스오픈프로모션_24.06-09대상자_241010_v3.xlsx', sheet_name=None)
    june = list(promotion.keys())[1]
    june_promotion = promotion[june]
    june_promotion = june_promotion.iloc[1:8,0:6]
    june_promotion['회원번호'] = june_promotion['회원번호'].astype(int)
    july = list(promotion.keys())[2]
    july_promotion = promotion[july]
    july_promotion = july_promotion.iloc[1:,0:6]
    #july_promotion = july_promotion.drop(411, axis=0)
    july_promotion['회원번호'] = july_promotion['회원번호'].astype(int)
    august = list(promotion.keys())[3]
    august_promotion = promotion[august]
    august_promotion = august_promotion.iloc[1:,0:6]
    august_promotion['회원번호'] = august_promotion['회원번호'].astype(int)
    september = list(promotion.keys())[4]
    september_promotion = promotion[september]
    september_promotion = september_promotion.iloc[1:,0:6]
    september_promotion['회원번호'] = september_promotion['회원번호'].astype(int)
    
    # june_promotion 제외
    #temp_merge = merge.merge(june_promotion[['회원번호', 'base_item_id 상품명']], 
    #                         left_on=['회원번호', 'plan'], 
    #                         right_on=['회원번호', 'base_item_id 상품명'], 
    #                         how='left', indicator='_merge1')
    #merge = temp_merge[temp_merge['_merge1'] == 'left_only'].drop(columns=['_merge1', 'base_item_id 상품명'])
    
    # july_promotion 제외
    #temp_merge = merge.merge(july_promotion[['회원번호', 'base_item_id 상품명']], 
    #                         left_on=['회원번호', 'plan'], 
    #                         right_on=['회원번호', 'base_item_id 상품명'], 
    #                          how='left', indicator='_merge2')
    #merge = temp_merge[temp_merge['_merge2'] == 'left_only'].drop(columns=['_merge2', 'base_item_id 상품명'])
    
    # august_promotion 제외
    temp_merge = merge.merge(august_promotion[['회원번호', 'base_item_id 상품명']], 
                             left_on=['회원번호', 'plan'],
                             right_on=['회원번호', 'base_item_id 상품명'], 
                             how='left', indicator='_merge3')
    merge = temp_merge[temp_merge['_merge3'] == 'left_only'].drop(columns=['_merge3', 'base_item_id 상품명'])
    
    # september_promotion 제외
    temp_merge = merge.merge(september_promotion[['회원번호', 'base_item_id 상품명']], 
                             left_on=['회원번호', 'plan'], 
                             right_on=['회원번호', 'base_item_id 상품명'],
                             how='left', indicator='_merge4')
    merge = temp_merge[temp_merge['_merge4'] == 'left_only'].drop(columns=['_merge4', 'base_item_id 상품명'])
    
    total_sales = merge['예상월매출'].sum() * month_day / i
    sales_expectation.append(total_sales)

In [106]:
sales_expectation11 = sales_expectation

In [107]:
merge.to_excel('/Users/user/Documents/업무문서/ncd11_sales_expectation.xlsx',index=False)

In [71]:
sales_expectation8

[2018744390.5433795,
 2018860738.058918,
 2018284306.5396001,
 2018236459.40642,
 2018726068.8043647,
 2019126112.9125135,
 2019496646.5127144,
 2019890564.5621457,
 2020359263.8340209,
 2020768974.639129,
 2021168190.2905989,
 2021589661.0016744,
 2021874053.8234816,
 2022186035.365587,
 2022479252.612583,
 2022854302.5584643,
 2023191097.8082607,
 2023619468.7040253,
 2024286226.5990894,
 2024974895.2258458,
 2025534819.6644099,
 2026099174.0416427,
 2026717964.7084537,
 2027362116.5368586,
 2027975520.0935733,
 2028613014.1684442,
 2029208777.022667,
 2029962719.80635,
 2030555798.5150564,
 2031173551.7632322,
 2031662017.5084133]

In [87]:
sales_expectation9

[2025918246.0195556,
 2028628704.784701,
 2029745289.931649,
 2029444715.7380922,
 2029857298.5072665,
 2030175299.2522964,
 2030386967.6182055,
 2030593376.595069,
 2031140706.5389638,
 2031394331.172871,
 2031713821.9342263,
 2032047220.5532885,
 2032802790.6078813,
 2033254473.9152257,
 2033661539.4369156,
 2034020601.650146,
 2034351522.4130838,
 2034644595.9827566,
 2034944822.2605915,
 2035450996.1986682,
 2035735807.8323274,
 2036034299.6958292,
 2036549053.362622,
 2037087854.870528,
 2037583197.742084,
 2038073814.4492939,
 2038657907.9069295,
 2039200143.7712386,
 2039760326.2807984,
 2040323138.3575335]

In [98]:
sales_expectation10

[2022525246.8856492,
 2022542498.5524793,
 2021711728.8927221,
 2021025216.8060112,
 2020669178.6698017,
 2020518936.4229262,
 2020891016.0476177,
 2021275852.8176444,
 2021553627.2532244,
 2021908034.6301875,
 2022299055.659404,
 2022443962.108876,
 2022873109.499788,
 2023272367.5638354,
 2023651722.014068,
 2023795634.9819505,
 2024144500.5646713,
 2024937902.7424533,
 2025049042.9425514,
 2025176037.1743667,
 2025449921.9819348,
 2025734193.341595,
 2026265302.4600337,
 2026604059.71719,
 2026935642.5599656,
 2027236972.951752,
 2027533246.7946126,
 2027856611.4427438,
 2028174700.4170732,
 2028566413.3252423,
 2028837767.7146783]

In [108]:
sales_expectation11

[2050579072.9558735,
 2050130723.8971918,
 2049673223.3492167,
 1537254917.5119126,
 1229803934.00953,
 1024836611.6746083,
 878431381.4353787,
 768627458.7559563,
 683224407.7830722,
 614901967.004765,
 559001788.1861501,
 512418305.8373042,
 473001513.08058846,
 439215690.71768934,
 409934644.6698434,
 384313729.37797815,
 361707039.41456765,
 341612203.8915361,
 323632614.2130342,
 307450983.5023825,
 292810460.47845954,
 279500894.09307504,
 267348681.30641958,
 256209152.9186521,
 245960786.80190602,
 236500756.54029423,
 227741469.2610241,
 219607845.35884467,
 212035161.03612587,
 204967322.3349217]